In [1]:
from keras.applications import VGG16 
conv_base = VGG16(weights='imagenet',
                 include_top=False, 
                 input_shape=(150, 150, 3))
conv_base.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [4]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [5]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
# data 



import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir= '/Users/developer/data/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)



Epoch 1/100
100/100 [==============================] - 668s 7s/step - loss: 0.5630 - acc: 0.7255 - val_loss: 0.3518 - val_acc: 0.8590
Epoch 2/100
100/100 [==============================] - 664s 7s/step - loss: 0.3965 - acc: 0.8270 - val_loss: 0.2581 - val_acc: 0.8910
Epoch 3/100
100/100 [==============================] - 657s 7s/step - loss: 0.3399 - acc: 0.8470 - val_loss: 0.2692 - val_acc: 0.8790
Epoch 4/100
100/100 [==============================] - 666s 7s/step - loss: 0.2939 - acc: 0.8670 - val_loss: 0.2262 - val_acc: 0.9040
Epoch 5/100
100/100 [==============================] - 662s 7s/step - loss: 0.2685 - acc: 0.8895 - val_loss: 0.1995 - val_acc: 0.9160
Epoch 6/100
 75/100 [=====================>........] - ETA: 1:57 - loss: 0.2498 - acc: 0.8980

In [ ]:

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

In [ ]:
model.save('cats_and_dogs_small_3.h5')

In [ ]:
from keras.models import load_model
model = load_model('cats_and_dogs_small_3.h5')
model.summary()

In [ ]:
img_path = '/Users/developer/data/dog.jpg'

from keras.preprocessing import image 
import numpy as np 

img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

print(img_tensor.shape)

In [ ]:
import matplotlib.pyplot as plt 

plt.imshow(img_tensor[0])
plt.show()

In [ ]:
from keras import models 

layer_outputs = [layer.output for layer in model.layers[:8]]
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(img_tensor)

first_layer_activation = activations[0]
print(first_layer_activation.shape)

plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')

In [ ]:
# visual the 7th channel 
plt.matshow(first_layer_activation[0, :, :, 7], cmap='viridis')

In [ ]:
# visualizing every channel in every intermediate activation
